In [30]:
"""
Name - Matrikelnummer 
1) Pham, Ngoc Anh Trung - 7176267
2) Viktor Vironski - 4330455
3) Andy Disser - 5984875

Exercise Sheet 2
"""

from random import random
from sklearn import cluster, datasets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl


class Node:
    """
    Node of a decision tree
    """

    def __init__(self, is_numerical, num_value, variable_index):
        self.left_child = None  # left child
        self.right_child = None  # right child
        self.is_numerical = is_numerical   # is numerical or not (i.e. is it a comparison of numerical)
        self.num_value = num_value  # numerical value (the threshold to be compared)
        self.variable_index = variable_index  # the index of the variable needs to be compared
        self.output_val = None  # The output value of this leaf for classification


def compute_gini_impurity(arr, labels, val):
    """
    Auxillary function for computing gini impurity value of the given array
    when using element with value val as root (numerical comparison and not binary comparison)

    Inputs:
    - arr : the given array
    - val : the given value
    - labels : the given labels
    Output:
    - the gini node impurity when using val as comparison value
    """

    # Only proceed if both the value is not nan and no entry in arr is nan
    if not (np.isinf(val)) and not (np.isinf(arr).any()):
        n = np.size(arr)
        left = arr[arr <= val]
        right = arr[arr > val]
        left_labels = labels[arr <= val]
        right_labels = labels[arr > val]

        # Yes : it belongs to class 0, No: it not belongs to class 0
        l = np.size(left)
        left_yes = np.size(left_labels[left_labels == 0])
        left_no = l - left_yes
        
        r = np.size(right)
        right_yes = np.size(right_labels[right_labels == 0])
        right_no = r - right_yes

        #1 - p(yes)**2 - p(no)**2
        left_impurity = 1 - np.power(np.divide(left_yes, l), 2) - np.power(np.divide(left_no, l), 2)

        if r != 0:
            # 1 - p(left)**2 - p(right)**2
            right_impurity = 1 - np.power(np.divide(right_yes, r), 2) - np.power(np.divide(right_no, r), 2)
        else:
            right_impurity = 0

        # node impurity : (#left/ #(left+right))*left_impurity + (#right/ #(left+right))*right_impurity
        node_impurity = np.multiply(np.divide(l, l+r), left_impurity) + np.multiply(np.divide(r, l+r), right_impurity)

        return node_impurity

    return np.infty


def decisiontree(data, labels, gini_impurity):
    """
    Build a decision tree based on the given data and labels

    Inputs:
    data : the given data
    labels : it's labels
    gini_impurity: initial gini impurity, usually inf

    Output:
    the root node of the decision tree
    """
    
    m = data.shape[1]  # the numbers of columns/ variables
    n = data.shape[0]  # the numbers of samples
    
    # transpose, for the sake of computing
    temp_data = np.transpose(data)

    # sort each row
    sorted_temp_data = np.sort(temp_data, axis=1)
    
    # Average of each adjacent pairs
    if n > 1:
        threshold_candidates = np.divide(np.delete(sorted_temp_data, n-1, axis=1)\
            +np.delete(sorted_temp_data, 0, axis=1), 2)
    else:
        threshold_candidates = sorted_temp_data

    # sort and take the sort indices each row of this tranposed matrix
    sort_indices = np.argsort(temp_data, axis=1)

    # permutate the labels (before that broadcast it to (m x n) matrix) accordingly
    # each row is a permuation of the labels according to the sorting of that row
    sorted_labels = np.take_along_axis(np.broadcast_to(labels[np.newaxis, :], (m, n)), sort_indices, 1)

    # Vectorize the function of computing gini impurity
    vec_compute_gini_impurity = np.vectorize(compute_gini_impurity, signature='(m), (m), () -> ()')

    # Impurity matrix , each entry (i, j) representing the gini impurity when using 
    # the entry (i, j) in sorted_temp_data as cutoff node.
    impurity_matrix = vec_compute_gini_impurity(sorted_temp_data[:, np.newaxis, :],\
        sorted_labels[:, np.newaxis, :], threshold_candidates)

    # lowest possible gini impurity of any sample and any variable
    min_gini_impurity = np.nanmin(np.apply_over_axes(np.nanmin, impurity_matrix, 1).reshape(m))

    # Variable index/row index that will be used to cut off
    var_index = np.nanargmin(np.apply_over_axes(np.nanmin, impurity_matrix, 1).reshape(m))

    #column index/ the index of the sample having the least impurity in this variable index
    samp_index = np.apply_over_axes(np.argmin, impurity_matrix, 1).reshape(m)[var_index]

    # The value of the cutoff node
    min_val = sorted_temp_data[var_index, samp_index]

    # Create a node containing the comparison
    node = Node(True, min_val, var_index)

    # Get the value of this specific variable
    min_data = data[:, var_index]

    # Put the samples than to the according left or right subtree
    left = data[min_data <= min_val]
    right = data[min_data > min_val]
    left_labels = labels[min_data <= min_val]
    right_labels = labels[min_data > min_val]

    # fill the already processed variable as infinity, causing it irrelevant for future computation
    left[:, var_index] = np.infty
    right[:, var_index] = np.infty

    if (np.all(np.isinf(data))) or (min_gini_impurity >= gini_impurity):
        # Base case, if the data only have inf left or
        # the gini impurity can not be any better

        # Then this node is a leaf, calculate the output value of this leaf based
        # on the the class that is more dominant in this leaf       
        if np.size(labels[labels == 0]) >= np.size(labels[labels == 1]):
            node.output_val = 0
        else:
            node.output_val = 1

        return node

    else:  # General case

        # Recurse for left and right subtree if there is any sample being assigned there
        if np.size(left) > 0:
            left_recurse = decisiontree(left, left_labels, min_gini_impurity)
            node.left_child = left_recurse  # Set the left child
        if np.size(right) > 0:
            right_recurse = decisiontree(right, right_labels, min_gini_impurity)
            node.right_child = right_recurse  # Set the right child

        return node
    
def predict(root, data):
    """
    Predict using the a decision tree

    data: the given data
    root: the root node of the given decision tree

    Output:
    0 or 1, 0 being the sample belongs to class 0, 1 being the sample not belong to class 0
    """

    curr_node = root  # current node

    # while it is not a leaf, has atleast one child
    while curr_node.left_child is not None or curr_node.right_child is not None:
        
        var_index = curr_node.variable_index
        
        # Left ("<="): the sample belongs to class 0, right (">"): the sample belongs to class 1 (not 0)
        if curr_node.left_child is not None:
            if curr_node.right_child is None:  # has left but doesn't have any right child
                curr_node = curr_node.left_child
            else:  # has both children
                if data[var_index] <= curr_node.num_value:
                    curr_node = curr_node.left_child
                else:  # data[var_index] > curr_node.num_value
                    curr_node = curr_node.right_child
        else:  # doesn't have any left child, but has a right child
            curr_node = curr_node.right_child

    return curr_node.output_val


def bootstrapdataset(data, nsamples):
    """
    Build bootstrapped dataset

    Inputs:
    - data: the given data
    - nsamples: the number of random samples with possible duplicate

    Output:
    - the bootstrapped dataset of the random samples with a subset of 3 features
    - the random indices with possbile duplicate
    """

    m = data.shape[1]  # the numbers of columns/ variables
    n = data.shape[0]  # the numbers of samples

    if nsamples == -1:
        rand_sample_indices = np.random.choice(np.arange(n), n, replace=True)
    else:
        rand_sample_indices = np.random.choice(np.arange(n), nsamples, replace=True)

    # Define the number of subset to be used
    num_of_features = 3

    # Only use this subset of the features
    features_subset = np.random.choice(np.arange(m), num_of_features, replace=False)

    # Filter the data based on the random indices and the subset of features
    data = data[:, features_subset]
    data = data[rand_sample_indices, :]

    return data, rand_sample_indices

In [35]:
# Import and access  data
wein = datasets.load_wine()
# print(wein.data.shape)  # 178, 13

# ==================== Exercise 1a : repeat 5 times ========================:

for _ in range(5):

    # Create permutation of 0,1,..,177
    permutation = np.random.permutation(np.arange(178))
    train_filter = permutation < 125
    test_filter = permutation >= 125 

    # Split the data, 125 samples (70%) for training, 53 samples (30%) for testing
    train_data = wein.data[train_filter]
    test_data = wein.data[test_filter]

    # The respective labels
    train_labels = wein.target[train_filter]
    test_labels = wein.target[test_filter]

    # merge classes bigger than 0 to 1 -> seperate 0 and not 0
    train_labels[train_labels > 0] = 1
    test_labels[test_labels > 0] = 1

    root = decisiontree(train_data, train_labels, np.infty)

    # vectorize and predict
    vec_predict = np.vectorize(predict, excluded='root', signature='(), (m) -> ()')
    predicted_labels = vec_predict(root, test_data)

    #print("predicted_labels:", predicted_labels)
    #print("truth labels:", test_labels)

    accuracy = np.sum(predicted_labels == test_labels) / np.size(test_labels)

    print(f'{_+1}. run, Accuracy: {accuracy}')

1. run, Accuracy: 0.9622641509433962
2. run, Accuracy: 0.9433962264150944
3. run, Accuracy: 0.9245283018867925
4. run, Accuracy: 0.8490566037735849
5. run, Accuracy: 0.9056603773584906


In [33]:
# ==================== Exercise 1b : build random forests ========================:

# Create permutation of 0,1,..,177
permutation = np.random.permutation(np.arange(178))
train_filter = permutation < 125
test_filter = permutation >= 125 

# Split the data, 125 samples (70%) for training, 53 samples (30%) for testing
train_data = wein.data[train_filter]
test_data = wein.data[test_filter]

# The respective labels
train_labels = wein.target[train_filter]
test_labels = wein.target[test_filter]

# merge classes bigger than 0 to 1 -> seperate 0 and not 0
train_labels[train_labels > 0] = 1
test_labels[test_labels > 0] = 1    

n = train_data.shape[0]   # Number of samples in train data
m = train_data.shape[1]   # number of features/variables in train data

num_of_trees = [10, 25, 100]

for k in num_of_trees:

    # Vetorize the bootstrapdataset function
    vec_bootstrapdataset= np.vectorize(bootstrapdataset, excluded='nsamples', signature='(m, n), () -> (k, l), (o)')

    # Broadcast the traindata and apply the vectorized bootstrapdataset on it
    broadcasted_train_data = np.broadcast_to(train_data, (k, n, m))
    bs_datasets, bs_random_indices = vec_bootstrapdataset(broadcasted_train_data, -1)

    # Extract the according labels
    broadcasted_train_labels = np.broadcast_to(train_labels, (k, n))
    bs_labels = np.take_along_axis(broadcasted_train_labels, bs_random_indices, axis=1)

    # build the forest
    vec_decisiontree = np.vectorize(decisiontree, excluded='gini_impurity', signature='(n, m), (n), () -> ()')
    forest = vec_decisiontree(bs_datasets, bs_labels, np.infty)

    # use the forest to predict, before that vectorize the predict function accordingly
    vec_predict = np.vectorize(predict, signature='(), (m) -> ()')  
    result = vec_predict(forest[np.newaxis, :], test_data[:, np.newaxis, :])

    # Sum over the result of each tree in forest, based on majority
    # if the sum (the number of 1) larger than 0, then the algorithm will
    # decide for 1 and vice versa
    mysum = np.apply_over_axes(np.sum, result, 1).reshape(test_data.shape[0])
    predicted = (mysum >= (k//2+1)).astype(int)

    # The final accuracy / Build the forest with 100 trees can take longer based on the size of the subset of features.
    accuracy = np.sum(predicted == test_labels) / np.size(test_labels)
    print(f'Forest with {k} trees, Accuracy: {accuracy}')

Forest with 10 trees, Accuracy: 0.6226415094339622
Forest with 25 trees, Accuracy: 0.6037735849056604
Forest with 100 trees, Accuracy: 0.5849056603773585
